In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [71]:
dataset=pd.read_csv('tweet_emotions.csv')

In [73]:
dataset.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [75]:
dataset['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [77]:
emotions_to_keep = ['happiness', 'sadness']

# Filter the dataset to keep only rows where 'sentiment' is either 'happy' or 'sad'
df = dataset[dataset['sentiment'].isin(emotions_to_keep)]

df.head()

,tweet_id,sentiment,content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?


In [79]:
df.shape

(10374, 3)

In [81]:
sad = df[df['sentiment'] == 'sadness']
happy = df[df['sentiment'] == 'happiness']

# to see if there are bias or not
num_sad = len(sad)
num_happy = len(happy)
print(num_sad)
print(num_happy)

5165
5209


In [83]:
columns_to_drop = ["tweet_id"]

df = df.drop(columns=columns_to_drop)

In [85]:
df['sentiment'], uniques = pd.factorize(df['sentiment'])

# Afficher les valeurs uniques et le DataFrame
print("Valeurs uniques :")
print(uniques)

Valeurs uniques :
Index(['sadness', 'happiness'], dtype='object')


In [87]:
df.head()

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [89]:
'''import re
import nltk
from nltk.corpus import stopwords

# Si ce n'est pas déjà fait, télécharger les stopwords de NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  # Utilise le français si les tweets sont en français, sinon 'english'

# Fonction de prétraitement du texte
def preprocess_tweet(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Supprimer les mentions et hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    # Supprimer la ponctuation
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)  # Supprimer les mots isolés
    text = re.sub(r'\s+', ' ', text)  # Remplacer les espaces multiples par un seul
    # Supprimer les stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Appliquer à la colonne 'content'
df['content'] = df['content'].apply(preprocess_tweet)'''


"import re\nimport nltk\nfrom nltk.corpus import stopwords\n\n# Si ce n'est pas déjà fait, télécharger les stopwords de NLTK\nnltk.download('stopwords')\nstop_words = set(stopwords.words('english'))  # Utilise le français si les tweets sont en français, sinon 'english'\n\n# Fonction de prétraitement du texte\ndef preprocess_tweet(text):\n    # Convertir en minuscules\n    text = text.lower()\n    # Supprimer les URL\n    text = re.sub(r'http\\S+|www\\S+|https\\S+', '', text, flags=re.MULTILINE)\n    # Supprimer les mentions et hashtags\n    text = re.sub(r'@\\w+|#\\w+', '', text)\n    # Supprimer la ponctuation\n    text = re.sub(r'\\s+[a-zA-Z]\\s+', ' ', text)  # Supprimer les mots isolés\n    text = re.sub(r'\\s+', ' ', text)  # Remplacer les espaces multiples par un seul\n    # Supprimer les stopwords\n    text = ' '.join([word for word in text.split() if word not in stop_words])\n    return text\n\n# Appliquer à la colonne 'content'\ndf['content'] = df['content'].apply(preprocess_t

In [91]:
df.head()

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
9,0,@kelcouch I'm sorry at least it's Friday?


In [205]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialiser le vectoriseur avec des paramètres personnalisés
vectorizer = TfidfVectorizer(
    max_features=2000,          # Limite à 8000 mots les plus fréquents
    max_df=0.9,                 # Exclure les termes apparaissant dans plus de 70% des documents
         # Appliquer une transformation logarithmique sur la fréquence des termes
)

# Appliquer le vectoriseur sur la colonne 'content'
X = vectorizer.fit_transform(df['content'])

# Afficher la dimension du tableau résultant
print(X.shape)  # (nombre d'échantillons, nombre de features)


(10374, 2000)


In [207]:
print(X[0])

  (0, 278)	0.41353629104877854
  (0, 1988)	0.2883331393447881
  (0, 1239)	0.2228607944168102
  (0, 1856)	0.559437552861706
  (0, 789)	0.4454321019564393
  (0, 1926)	0.2386698401138877
  (0, 181)	0.35754912595039634


In [209]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
y = df['sentiment']  # Sentiments associés aux tweets (0 ou 1)

# Diviser les données en ensemble d'entraînement et de test (80% entraînement, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialiser le classificateur Naive Bayes
model = MultinomialNB()

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)
# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)
y_pred2 = model.predict(X_train)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_train, y_pred2)

print(f'Précision du modèle Naive Bayes: {accuracy * 100:.2f}%')
print(f'Précision du modèle Naive Bayes train: {accuracy2 * 100:.2f}%')

Précision du modèle Naive Bayes: 79.23%
Précision du modèle Naive Bayes train: 84.93%


In [211]:
from sklearn.linear_model import LogisticRegression
# Initialiser la régression logistique avec une régularisation bayésienne (C est l'inverse de la régularisation)
model = LogisticRegression(C=1.01, solver='lbfgs', max_iter=10000)

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)
# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)
y_pred2 = model.predict(X_train)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_train, y_pred2)

print(f'Précision du modèle Logistic Regression: {accuracy * 100:.2f}%')
print(f'Précision du modèle Logistic Regression train: {accuracy2 * 100:.2f}%')


Précision du modèle Logistic Regression: 80.96%
Précision du modèle Logistic Regression train: 86.35%


In [ ]:
# Initialize the SVM model
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', gamma='scale') #gaussian


# Train the model
svm_model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred2 = model.predict(X_train)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_train, y_pred2)

print(f'Précision du modèle SVM: {accuracy * 100:.2f}%')
print(f'Précision du modèle SVM train: {accuracy2 * 100:.2f}%')

In [203]:
from sklearn.neighbors import KNeighborsClassifier

# Initialiser le modèle KNN
model = KNeighborsClassifier(n_neighbors=2)

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)
y_pred2 = model.predict(X_train)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_train, y_pred2)

print(f'Précision du modèle K-neighbours: {accuracy * 100:.2f}%')
print(f'Précision du modèle K-neighbours: {accuracy2 * 100:.2f}%')


Précision du modèle K-neighbours: 52.72%
Précision du modèle K-neighbours: 98.36%


In [147]:
from sklearn.ensemble import RandomForestClassifier

# Initialiser le modèle de forêt aléatoire
model = RandomForestClassifier(n_estimators=100)

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)
y_pred2 = model.predict(X_train)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_train, y_pred2)

print(f'Précision du modèle Forest: {accuracy * 100:.2f}%')
print(f'Précision du modèle Forest: {accuracy2 * 100:.2f}%')


Précision du modèle K-neighbours: 78.41%
Précision du modèle K-neighbours: 99.87%


In [189]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialiser le modèle de gradient boosting
model = GradientBoostingClassifier(n_estimators=100)

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)
y_pred2 = model.predict(X_train)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_train, y_pred2)

print(f'Précision du modèle GBC: {accuracy * 100:.2f}%')
print(f'Précision du modèle GBC: {accuracy2 * 100:.2f}%')

Précision du modèle GBC: 74.89%
Précision du modèle GBC: 78.55%
